In [ ]:
import pandas as pd
import numpy as np
import torch
from scipy.spatial import distance_matrix
import numpy as np
import tensorflow
import spektral
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from spektral.data import Dataset, BatchLoader, Graph
from spektral.transforms.normalize_adj import NormalizeAdj
from tensorflow.keras.layers import Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [60]:

df = pd.read_csv('train.csv', header=[0, 1])
test_raw = pd.read_csv('test.csv', header=[0, 1])
meta_df = pd.read_csv("metaData.csv")
#get target feature out of df
target = df[['Timestamp',"target_feature"]].rename(columns={"Unnamed: 1_level_1":"","Unnamed: 53_level_1":""})
target.columns = [''.join(col).strip() for col in target.columns] 
target.drop(target.iloc[0].name, inplace=True)
target["Timestamp"] = pd.to_datetime(target["Timestamp"])
df.head()

C:\Users\conny\AppData\Local\Temp\ipykernel_10104\1739839201.py:1: DtypeWarning: Columns (0,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv', header=[0, 1])
C:\Users\conny\AppData\Local\Temp\ipykernel_10104\1739839201.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_raw = pd.read_csv('test.csv', header=[0, 1])


Unnamed: 0_level_0            Timestamp Wind speed (m/s)             \
             Turbine   Unnamed: 1_level_1       Kelmarsh 2 Kelmarsh 3   
0                 id                  NaN              NaN        NaN   
1                  0  2017-10-01 00:00:00         9.112875   9.197829   
2                  1  2017-10-01 00:10:00         8.319863   9.350296   
3                  2  2017-10-01 00:20:00         8.799176   8.510402   
4                  3  2017-10-01 00:30:00         8.392921   8.825861   

                                   Wind speed, Standard deviation (m/s)  \
  Kelmarsh 4 Kelmarsh 5 Kelmarsh 6                           Kelmarsh 2   
0        NaN        NaN        NaN                                  NaN   
1   8.795919   8.749384   7.684954                             1.104222   
2   8.762059   8.652318   7.421576                             1.207655   
3   9.533566   8.166586   7.797040                             1.124029   
4   8.990985   7.877046   7.750090                             1.219647   

                         ... Generator RPM (RPM)                            \
  Kelmarsh 3 Kelmarsh 4  ...          Kelmarsh 4   Kelmarsh 5   Kelmarsh 6   
0        NaN        NaN  ...                 NaN          NaN          NaN   
1   1.096619   0.999588  ...         1768.215088  1733.296875  1603.749390   
2   0.916610   0.956899  ...         1770.806885  1737.218262  1572.769409   
3   0.992841   1.343180  ...         1766.002319  1698.699219  1655.049316   
4   1.218454   0.970566  ...         1783.569824  1661.264526  1678.571777   

  Blade angle (pitch position) A (°)                                   \
                          Kelmarsh 2 Kelmarsh 3 Kelmarsh 4 Kelmarsh 5   
0                                NaN        NaN        NaN        NaN   
1                           0.183487   0.135490   0.001053     0.0000   
2                           0.051998   0.086999   0.095998     0.0605   
3                           0.168988   0.056997   0.087998     0.0090   
4                           0.084492   0.133995   0.017500     0.0000   

                        training      target_feature  
  Kelmarsh 6 Unnamed: 52_level_1 Unnamed: 53_level_1  
0        NaN                 NaN                 NaN  
1      0.000                True            8.602844  
2      0.000                True            8.125226  
3      0.016                True            7.551986  
4      0.000                True            7.912114  

[5 rows x 54 columns]

In [61]:
df.columns = ['_'.join(col).strip() for col in df.columns]  # Flatten multi-index
df.rename(columns={"Timestamp_Unnamed: 1_level_1": "Timestamp"}, inplace=True)  # Rename timestamp
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
test_raw.columns = ['_'.join(col).strip() for col in test_raw.columns]  # Flatten multi-index
test_raw.rename(columns={"Timestamp_Unnamed: 1_level_1": "Timestamp"}, inplace=True)  # Rename timestamp
test_raw["Timestamp"] = pd.to_datetime(test_raw["Timestamp"])

In [62]:
df_long = df.melt(id_vars=["Timestamp"], 
                  var_name="Variable_Turbine", 
                  value_name="Value")

# Split 'Variable_Turbine' into 'Variable' and 'Turbine'
df_long[["Variable", "Turbine"]] = df_long["Variable_Turbine"].str.rsplit("_", n=1, expand=True)
df_long.drop(columns=["Variable_Turbine"], inplace=True)
df_long=df_long[df_long["Turbine"].str.contains("Kelmarsh")]
df_long.head()  # View the reshaped dataframedf
test_long = test_raw.melt(id_vars=["Timestamp"], 
                  var_name="Variable_Turbine", 
                  value_name="Value")

# Split 'Variable_Turbine' into 'Variable' and 'Turbine'
test_long[["Variable", "Turbine"]] = test_long["Variable_Turbine"].str.rsplit("_", n=1, expand=True)
test_long.drop(columns=["Variable_Turbine"], inplace=True)
test_long=test_long[test_long["Turbine"].str.contains("Kelmarsh")]

In [63]:
df_wide = df_long.pivot(index=['Timestamp', 'Turbine'], columns='Variable', values='Value').reset_index()
df_wide = df_wide[df_wide["Timestamp"].notna()]
test_wide = test_long.pivot(index=['Timestamp', 'Turbine'], columns='Variable', values='Value').reset_index()
test_wide = test_wide[test_wide["Timestamp"].notna()]

In [64]:
meta_df["total_hub_height"] = meta_df["Hub Height (m)"]  + meta_df["Elevation (m)"] 

In [65]:
#df_wide_meta = df_wide.merge(meta_df, left_on="Turbine",right_on="Title")

In [66]:
df_final = df_wide.merge(target, on="Timestamp")

In [67]:
df_final.columns

Index(['Timestamp', 'Turbine', 'Blade angle (pitch position) A (°)',
       'Generator RPM (RPM)', 'Nacelle ambient temperature (°C)',
       'Nacelle position (°)', 'Nacelle position, Standard deviation (°)',
       'Power (kW)', 'Vane position 1+2 (°)', 'Wind direction (°)',
       'Wind speed (m/s)', 'Wind speed, Standard deviation (m/s)',
       'target_feature'],
      dtype='object')

In [68]:
def get_turbine_distances(meta_df):
    """
    Compute pairwise distances between turbines.

    Args:
        meta_df (pd.DataFrame): DataFrame containing 'Latitude' and 'Longitude'.

    Returns:
        array: {("Turbine A", "Turbine B"): (distance)}
        
    """
    coords = meta_df[['Latitude', 'Longitude']].values
    
    # Calculate pairwise distances
    dist_matrix = distance_matrix(coords, coords)
    
    return dist_matrix

In [69]:
turbine_distances = get_turbine_distances(meta_df)

In [70]:
def get_turbine_features(df):
    """
    get features for turbines and order by turbine name

    Args:
        df (pd.DataFrame): DataFrame containing features for a timestamp
        

    Returns:
        
    """
    features = df.sort_values(by="Turbine")
    features = features.drop(["Timestamp","Turbine","target_feature"],axis=1, errors='ignore')

    return features

In [ ]:
unique_timestamps = df_final['Timestamp'].unique()  # replace 'timestamp' with your actual column name
np.random.shuffle(unique_timestamps)  # shuffle in-place

# Calculate split index (25% for validation)
split_idx = int(len(unique_timestamps) * 0.75)

# Split timestamps
train_timestamps = unique_timestamps[:split_idx]
val_timestamps = unique_timestamps[split_idx:]

# Create train/validation DataFrames
train_df = df_final[df_final['Timestamp'].isin(train_timestamps)]
val_df = df_final[df_final['Timestamp'].isin(val_timestamps)]



In [73]:
def get_data_graph_split(df, train=True):
    data_split = {"train_x": [],"train_a": [],"y":[]}
    for timestamp in df.Timestamp.unique():
        subset = df[df.Timestamp==timestamp]
        data_split["train_x"].append(get_turbine_features(subset))
        data_split["train_a"].append(turbine_distances)
        if train==True:
            data_split["y"].append(subset.target_feature.head(1))
    
    return data_split

In [ ]:
local_train = get_data_graph_split(train_df,True)
local_val = get_data_graph_split(val_df, False)
local_test = get_data_graph_split(test_wide,False)

In [ ]:
################################################################################
# Config
################################################################################
learning_rate = 1e-2  # Learning rate
epochs = 100  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 21  # Batch size


In [ ]:
################################################################################
# Load data
################################################################################

class GraphDataset(Dataset):
    def __init__(self, n_samples, df, n_colors=120, **kwargs):
        self.n_samples = n_samples
        self.df = df  
        self.n_colors = n_colors  
        super().__init__(**kwargs)

    def read(self):
        output = []
        for i in range(self.n_samples):
            # Node features
            iter_x = self.df["train_x"][i].copy()
            x = np.array(iter_x).reshape(5,10)

            # Edges
            iter_a =  self.df["train_a"][i].copy()
            a = np.array(iter_a).reshape(6,6)

            # 
            y = np.zeros((24,))
            y_index = int(self.df["y"][i])
            y[y_index:] = 1
           
            output.append(Graph(x=x, a=a, y=y))
            print(i)
        return(output)

SyntaxError: '(' was never closed (1250529089.py, line 17)

In [ ]:

# Train/valid/test split
len_train = len(local_train["train_x"])
len_val = len(local_val["train_x"])
len_test = len(local_test["train_x"])
data_tr = GraphDataset(n_samples = len_train, df = local_train, transforms=NormalizeAdj())
data_val = GraphDataset(n_samples = len_val, df = local_val, transforms=NormalizeAdj())
data_te = GraphDataset(n_samples = len_test, df=local_test, transforms=NormalizeAdj())
# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_val = BatchLoader(data_val, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)


In [ ]:
################################################################################
# Build model
################################################################################
class BDB22GNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'sigmoid')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

In [ ]:

model = BDB22GNN(21, 120)
model.compile('adam', "mean_absolute_error")

model.fit(loader_tr.load(), validation_data= loader_val.load(), steps_per_epoch=loader_tr.steps_per_epoch,validation_steps=loader_val.steps_per_epoch,
     epochs=epochs)#
test_loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)

print('Test loss: {}'.format(test_loss))
  

predictions = model.predict(loader_te.load(), steps =loader_te.steps_per_epoch)

